In [2]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model, StatsTracker
from otgnn.graph import SYMBOLS, FORMAL_CHARGES, BOND_TYPES, get_bt_index

from mol_opt.mol_opt import MolOpt
from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.decoder_mol_opt import MolOptDecoder
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model
from mol_opt.ot_utils import encode_target, FGW
from mol_opt.train_mol_opt import ft, load_checkpoint, save_checkpoint, get_latest_model, initialize_models

from rdkit.Chem import MolFromSmiles

from molgen.metrics.Penalty import Penalty

import torch
from torch import nn
import numpy as np
import time
import shutil

/home/octav/gitrepos/tum-thesis


In [3]:
def rename(oldname, newname):
    wd = "mol_opt/output_dev3/"
    oldwd = wd

    olddir = os.path.join(oldwd, oldname)
    newdir = os.path.join(wd, newname)

    shutil.copytree(olddir, newdir)

    print (newdir)
    for fl in os.listdir(newdir):
    #     print (fl)
        newfl = fl.replace(oldname, newname)
        os.rename(os.path.join(wd, newname, fl), os.path.join(wd, newname, newfl))
        
    wd = "mol_opt/logs_dev3/"
    olddir = os.path.join(wd, oldname)
    newdir = os.path.join(wd, newname)

    shutil.copytree(olddir, newdir)

In [4]:
oldname = ""
newname = "molemb-base-nopen-softmax"
rename(oldname, newname)

FileExistsError: [Errno 17] File exists: 'mol_opt/output_dev3/molemb-base-nopen-softmax'

In [25]:
oldname = "pointwise-base-nopen-4000-softmax4"
newname = "pointwise-base-longer-softmax3"
rename(oldname, newname)

In [56]:
wd = "mol_opt/output_dev2/"
newname = "pointwise-base-longer-softmax3"
filename = os.path.join(wd, newname, "model_" + newname + "_12009")


('model_pointwise-base-longer-softmax3_12009', 12009)

In [5]:
wd = "mol_opt/output_dev3/"
newname = "deepsets-qed1"
filename, _ = get_latest_model(newname, os.path.join(wd, newname))
print (filename)
filepath = os.path.join(wd, newname, filename)
molopt, molopt_decoder, optimizer, penalty, recpen, crossatt, scheduler, args, epoch = load_checkpoint(filepath, initialize_models)

args.connectivity = True 
args.valency = True 
args.euler_characteristic_penalty = True
# args.conn_lambda_start = 0.89
# args.conn_lambda_end = 1
args.conn_lambda_epochs_start = 12
args.conn_lambda_epochs_end = 20
# args.valency_lambda_start = 2.2
# args.valency_lambda_end = 3
args.valency_lambda_epochs_start = 12
args.valency_lambda_epochs_end = 20
# args.euler_lambda_start = 1.4
# args.euler_lambda_end = 2 
args.euler_lambda_epochs_start = 12
args.euler_lambda_epochs_end = 20
args.tau_start = 1
args.tau_end = 0.01
args.tau_epochs_start = 1
args.tau_epochs_end = 16


penalty.conn_lambda = args.conn_lambda_start
penalty.valency_lambda = args.valency_lambda_start
penalty.euler_lambda = args.euler_lambda_start

save_checkpoint(molopt, molopt_decoder, optimizer, penalty, recpen, crossatt, scheduler, epoch, args, filepath)
molopt, molopt_decoder, optimizer, penalty, recpen, crossatt, scheduler, args, epoch = load_checkpoint(filepath, initialize_models)
penalty.log()
penalty.euler_lambda_epochs_start
# penalty.connectivity
print (penalty.conn_lambda_epochs_start)
print (penalty.conn_lambda_epochs_end)
print (penalty.tau_epochs_end)
print(penalty.conn_lambda_rate)
print(penalty.euler_lambda_rate)

model_deepsets-qed1_4
Penalty params: tau=0.28480 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=4 mode=[0 0 0] conn=True val=True euler=True
12
20
16
3.1385684721559213
2.9619362959451734


In [4]:
wd = "mol_opt/output_dev4/"
newname = "deepsets-noatt-base1"
filename, _ = get_latest_model(newname, os.path.join(wd, newname))
print (filename)
filepath = os.path.join(wd, newname, filename)
molopt, molopt_decoder, optimizer, penalty, recpen, crossatt, scheduler, args, epoch = load_checkpoint(filepath, initialize_models, device = 'cpu')


# args.scale_lambdas = True
args.connectivity = True 
args.valency = True 
args.euler_characteristic_penalty = True
# args.reconstruction_loss = True
# args.conn_lambda_start = 0.001
# args.conn_lambda_end = 1
# args.conn_lambda_epochs_start = 5000
# args.conn_lambda_epochs_end = 13000
# args.valency_lambda_start = 0.001
# args.valency_lambda_end = 3
# args.valency_lambda_epochs_start = 5000
# args.valency_lambda_epochs_end = 13000
# args.euler_lambda_start = 0.001 
# args.euler_lambda_end = 2
# args.euler_lambda_epochs_start = 5000
# args.euler_lambda_epochs_end = 13000
# args.rec_lambda_start = 0.001 
# args.rec_lambda_end = 5
# args.rec_lambda_epochs_start = 4000
# args.rec_lambda_epochs_end = 12000
# args.tau_start = 1
# args.tau_end = 0.01
# args.tau_epochs_start = 1
# args.tau_epochs_end = 12000
# args.device = 'cpu'
# args.batch_size = 500


# penalty.conn_lambda = args.conn_lambda_start
# penalty.valency_lambda = args.valency_lambda_start
# penalty.euler_lambda = args.euler_lambda_start

save_checkpoint(molopt, molopt_decoder, optimizer, penalty, recpen, crossatt, scheduler, epoch, args, filepath)
molopt, molopt_decoder, optimizer, penalty, recpen, crossatt, scheduler, args, epoch = load_checkpoint(filepath, initialize_models)
penalty.log()
penalty.euler_lambda_epochs_start
# penalty.connectivity
penalty.tau_epochs_end
print(penalty.conn_lambda_rate)
print(penalty.euler_lambda_rate)
print(recpen.rec_lambda_rate)

model_deepsets-noatt-base1_4000
Penalty params: tau=0.21550 conn_l=0.00100 val_l=0.00100 euler_l=0.00100 epoch=4000 mode=[0 0 0] conn=True val=True euler=True
1.001001547421495
1.0009508021210662
1.0985411419875581


In [100]:
model_molemb-base-longer-softmax5_4001
Penalty params: tau=0.17773 conn_l=1.00000 val_l=10.00000 euler_l=1.00000 epoch=4001 mode=[0 0 0] conn=False euler=False val=False
1.0017020455288321
1.0014989884985714


1.001702174898471

In [101]:
1.05201/1.05043

1.0015041459211942

In [4]:
wd = "mol_opt/output_dev2/"
oldname = "pointwise-base-nopen-4000-gumbel4"
dirname = os.path.join(wd, oldname)

In [15]:
os.listdir(dirname)

['model_pointwise-base-nopen-4000-gumbel4_1000',
 'model_pointwise-base-nopen-4000-gumbel4_2000',
 'model_pointwise-base-nopen-4000-gumbel4_3000',
 'model_pointwise-base-nopen-4000-gumbel4_3100',
 'model_pointwise-base-nopen-4000-gumbel4_3200',
 'model_pointwise-base-nopen-4000-gumbel4_3300',
 'model_pointwise-base-nopen-4000-gumbel4_3400',
 'model_pointwise-base-nopen-4000-gumbel4_3500',
 'model_pointwise-base-nopen-4000-gumbel4_3600',
 'model_pointwise-base-nopen-4000-gumbel4_3700',
 'model_pointwise-base-nopen-4000-gumbel4_3800',
 'model_pointwise-base-nopen-4000-gumbel4_3900',
 'model_pointwise-base-nopen-4000-gumbel4_3910',
 'model_pointwise-base-nopen-4000-gumbel4_3920',
 'model_pointwise-base-nopen-4000-gumbel4_3930',
 'model_pointwise-base-nopen-4000-gumbel4_3940',
 'model_pointwise-base-nopen-4000-gumbel4_3950',
 'model_pointwise-base-nopen-4000-gumbel4_3960',
 'model_pointwise-base-nopen-4000-gumbel4_3970',
 'model_pointwise-base-nopen-4000-gumbel4_3980',
 'model_pointwise-ba

In [8]:
_, lastepoch = get_latest_model(oldname, dirname)

In [14]:
for fl in os.listdir(dirname):
    ep = int(fl.split("_")[2])
    ep_diff = lastepoch - ep
    for modulo in [10, 100, 1000]:
        if ep_diff > modulo and ep % modulo != 0:
            os.remove(os.path.join(dirname, fl))